In [1]:
import pandas as pd
import numpy as np
from numpy import inf, mean, median
import math
import datetime
import json
import os
import glob
import pickle
import requests
# import requests_cache
import plotly.graph_objects as go
import plotly.express as px
import folium
from folium.plugins import MarkerCluster, HeatMap
import geopandas as gpd
import osm2geojson
import geojson
import shapely
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString

# Load Data

In [34]:
data_mos_token = '7f7ec5c82e28b89080babbc1c5aa5ab9'

In [35]:
data_mos_dataset_id = 624

In [36]:
data_mos_request = f'https://apidata.mos.ru/v1/datasets/{data_mos_ru_dataset_id}/features?api_key={data_mos_ru_token}'
data_mos_request

'https://apidata.mos.ru/v1/datasets/624/features?api_key=7f7ec5c82e28b89080babbc1c5aa5ab9'

In [43]:
data_mos_get = requests.get('https://apidata.mos.ru/v1/datasets/624/features?api_key=7f7ec5c82e28b89080babbc1c5aa5ab9')

In [51]:
data_mos_metro_entrances_gj = data_mos_get.json()

In [53]:
data_mos_metro_entrances_gj['features'][0]

{'geometry': {'coordinates': [37.6316766, 55.7573154], 'type': 'Point'},
 'properties': {'DatasetId': 624,
  'VersionNumber': 4,
  'ReleaseNumber': 98,
  'RowId': None,
  'Attributes': {'Name': 'Китай-город, вход-выход 5 в северный вестибюль',
   'Longitude_WGS84': '37.6316766',
   'Latitude_WGS84': '55.7573154',
   'NameOfStation': 'Китай-город',
   'Line': 'Калужско-Рижская линия',
   'ModeOnEvenDays': 'открытие в 05:30:00; закрытие в 01:00:00; первый поезд по 1 пути в 05:44:00, по 2 пути в 05:49:05; последний поезд по 1 пути в 01:36:20, по 2 пути в 01:27:24',
   'ModeOnOddDays': 'открытие в 05:30:00; закрытие в 01:00:00; первый поезд по 1 пути в 05:43:05, по 2 пути в 05:49:50; последний поезд по 1 пути в 01:36:20, по 2 пути в 01:27:24',
   'FullFeaturedBPAAmount': 0,
   'LittleFunctionalBPAAmount': 4,
   'BPAAmount': 4,
   'RepairOfEscalators': [],
   'global_id': 1773539}},
 'type': 'Feature'}

In [13]:
len(data_mos_metro_entrances_gj['features'])

1009

In [56]:
with open('data_mos_metro_entrances_gj.geojson', 'w') as _f:
        geojson.dump(data_mos_metro_entrances_gj, _f)

# Prepare Data

In [7]:
with open('data_mos_metro_entrances_gj.geojson', 'r') as _f:
        data_mos_metro_entrances_gj = geojson.load(_f)

In [8]:
type(data_mos_metro_entrances_gj)

geojson.feature.FeatureCollection

In [9]:
data_mos_metro_entrances_gj['features'][0]

{"geometry": {"coordinates": [37.631677, 55.757315], "type": "Point"}, "properties": {"Attributes": {"BPAAmount": 4, "FullFeaturedBPAAmount": 0, "Latitude_WGS84": "55.7573154", "Line": "\u041a\u0430\u043b\u0443\u0436\u0441\u043a\u043e-\u0420\u0438\u0436\u0441\u043a\u0430\u044f \u043b\u0438\u043d\u0438\u044f", "LittleFunctionalBPAAmount": 4, "Longitude_WGS84": "37.6316766", "ModeOnEvenDays": "\u043e\u0442\u043a\u0440\u044b\u0442\u0438\u0435 \u0432 05:30:00; \u0437\u0430\u043a\u0440\u044b\u0442\u0438\u0435 \u0432 01:00:00; \u043f\u0435\u0440\u0432\u044b\u0439 \u043f\u043e\u0435\u0437\u0434 \u043f\u043e 1 \u043f\u0443\u0442\u0438 \u0432 05:44:00, \u043f\u043e 2 \u043f\u0443\u0442\u0438 \u0432 05:49:05; \u043f\u043e\u0441\u043b\u0435\u0434\u043d\u0438\u0439 \u043f\u043e\u0435\u0437\u0434 \u043f\u043e 1 \u043f\u0443\u0442\u0438 \u0432 01:36:20, \u043f\u043e 2 \u043f\u0443\u0442\u0438 \u0432 01:27:24", "ModeOnOddDays": "\u043e\u0442\u043a\u0440\u044b\u0442\u0438\u0435 \u0432 05:30:00; \u0437

In [15]:
len(data_mos_metro_entrances_gj['features'])

1009

In [10]:
data_mos_metro_entrances_gdf = gpd.GeoDataFrame.from_features(data_mos_metro_entrances_gj['features'])

In [11]:
data_mos_metro_entrances_gdf.head()

,geometry,DatasetId,VersionNumber,ReleaseNumber,RowId,Attributes
0,POINT (37.63168 55.75731),624,4,98,None,"{'Name': 'Китай-город, вход-выход 5 в северный..."
1,POINT (37.63121 55.75686),624,4,98,None,"{'Name': 'Китай-город, вход-выход 4 в северный..."
2,POINT (37.63195 55.75724),624,4,98,None,"{'Name': 'Китай-город, вход-выход 6 в северный..."
3,POINT (37.63308 55.75310),624,4,98,None,"{'Name': 'Китай-город, вход-выход 14 в южный в..."
4,POINT (37.63356 55.75307),624,4,98,None,"{'Name': 'Китай-город, вход-выход 12 в южный в..."


In [12]:
data_mos_metro_entrances_gdf.shape

(1009, 6)

In [16]:
tags_to_parce = [
    'Name',
    'NameOfStation',
    'Line',
    'global_id',
]

In [17]:
for tag in tags_to_parce:
    data_mos_metro_entrances_gdf[tag] = data_mos_metro_entrances_gdf['Attributes'].apply(lambda x: x.get(tag) if tag in x else None)

In [18]:
data_mos_metro_entrances_gdf.head()

,geometry,DatasetId,VersionNumber,ReleaseNumber,RowId,Attributes,Name,NameOfStation,Line,global_id
0,POINT (37.63168 55.75731),624,4,98,None,"{'Name': 'Китай-город, вход-выход 5 в северный...","Китай-город, вход-выход 5 в северный вестибюль",Китай-город,Калужско-Рижская линия,1773539
1,POINT (37.63121 55.75686),624,4,98,None,"{'Name': 'Китай-город, вход-выход 4 в северный...","Китай-город, вход-выход 4 в северный вестибюль",Китай-город,Калужско-Рижская линия,1773540
2,POINT (37.63195 55.75724),624,4,98,None,"{'Name': 'Китай-город, вход-выход 6 в северный...","Китай-город, вход-выход 6 в северный вестибюль",Китай-город,Калужско-Рижская линия,1773541
3,POINT (37.63308 55.75310),624,4,98,None,"{'Name': 'Китай-город, вход-выход 14 в южный в...","Китай-город, вход-выход 14 в южный вестибюль",Китай-город,Калужско-Рижская линия,1773542
4,POINT (37.63356 55.75307),624,4,98,None,"{'Name': 'Китай-город, вход-выход 12 в южный в...","Китай-город, вход-выход 12 в южный вестибюль",Китай-город,Калужско-Рижская линия,1773543


In [19]:
with open('data_mos_metro_entrances_gdf.pickle', 'wb') as _f:
    pickle.dump(data_mos_metro_entrances_gdf, _f, pickle.HIGHEST_PROTOCOL)

# Group Entrances To Stations

In [ ]:
with open('data_mos_metro_entrances_gdf.pickle', 'rb') as _f:
    data_mos_metro_entrances_gdf = pickle.load(_f)

In [25]:
data_mos_metro_stations_gdf = data_mos_metro_entrances_gdf[['geometry', 'NameOfStation', 'Line']].dissolve(by=['NameOfStation', 'Line']).reset_index()

In [26]:
data_mos_metro_stations_gdf.head()

,NameOfStation,Line,geometry
0,Авиамоторная,Большая кольцевая линия,POINT (37.71948 55.75325)
1,Авиамоторная,Калининская линия,"MULTIPOINT (37.71637 55.75168, 37.71640 55.751..."
2,Автозаводская,Замоскворецкая линия,"MULTIPOINT (37.65732 55.70643, 37.65737 55.708..."
3,Автозаводская,Московское центральное кольцо,"MULTIPOINT (37.66057 55.70599, 37.66290 55.70537)"
4,Академическая,Калужско-Рижская линия,"MULTIPOINT (37.57222 55.68702, 37.57237 55.687..."


In [27]:
data_mos_metro_stations_gdf.rename(columns={'NameOfStation':'station_name', 'Line':'line_name'}, inplace=True)

In [28]:
data_mos_metro_stations_gdf['station_point'] = data_mos_metro_stations_gdf['geometry'].representative_point()

In [29]:
data_mos_metro_stations_gdf.head()

,station_name,line_name,geometry,station_point
0,Авиамоторная,Большая кольцевая линия,POINT (37.71948 55.75325),POINT (37.71948 55.75325)
1,Авиамоторная,Калининская линия,"MULTIPOINT (37.71637 55.75168, 37.71640 55.751...",POINT (37.71680 55.75196)
2,Автозаводская,Замоскворецкая линия,"MULTIPOINT (37.65732 55.70643, 37.65737 55.708...",POINT (37.65737 55.70860)
3,Автозаводская,Московское центральное кольцо,"MULTIPOINT (37.66057 55.70599, 37.66290 55.70537)",POINT (37.66057 55.70599)
4,Академическая,Калужско-Рижская линия,"MULTIPOINT (37.57222 55.68702, 37.57237 55.687...",POINT (37.57306 55.68695)


In [10]:
data_mos_metro_stations_gdf.shape

(264, 4)

In [30]:
with open('data_mos_metro_stations_gdf.pickle', 'wb') as _f:
    pickle.dump(data_mos_metro_stations_gdf, _f, pickle.HIGHEST_PROTOCOL)

In [6]:
with open('data_mos_metro_stations_gdf.pickle', 'rb') as _f:
    data_mos_metro_stations_gdf = pickle.load(_f)